# Workshop Part 2 of 2

At the beginning of the workshop, you were given a room name and workshop key.

Type those in the following cell, and then run the cell to assign them to global variables to be used throughout the rest of this notebook.

In [ ]:
# Cell: 01

g_room_name = ""
g_workshop_key = ""

# Table of contents

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>Step 7:</b> Catch up from Part 1 of the workshop</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 8:</b> Customize NLP model for domain using dictionaries</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 9:</b> Identify entities in sticky note comments</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 10:</b> Cluster sticky notes by entities</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 11:</b> Add positional data to improve clustering</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 12:</b> Visualize clusters in the mural</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 13:</b> Add a chart to the mural</li>
</ol>

# Step 7: Catch up from Part 1 of the workshop

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>7.1</b> Create a mural with sample sticky notes</li>
<li style="margin: 0px 0px 3px 0px;"><b>7.2</b> Put sticky note data into a DataFrame</li>
<li style="margin: 0px 0px 3px 0px;"><b>7.3</b> Run NLP analysis on sticky note comments</li>
</ol>

## 7.1 Create a mural with sample sticky notes

In [ ]:
# Cell: 02

import requests
import json
from IPython.display import display, HTML

def setUpForPart2( room_name, workshop_key ):
    url = "https://weavesphere-mural-oauth.tqns6lm651z.us-south.codeengine.appdomain.cloud/create-mural-workshop-part2"
    headers = { "Content-Type" : "application/json", "Accept" : "application/json" }
    data = json.dumps( { "room_name" : room_name, "workshop_key" : workshop_key } )
    response = requests.request( "POST", url, headers=headers, data=data )
    response_json = response.json()
    if "error_str" in response_json:
        print( response_json["error_str"] )
        return None
    if "mural_id" not in response_json:
        print( "Field 'mural_id' not returned in result\nResult:" + json.dumps( response_json, indent=3 ) )
        return None
    if "mural_link" not in response_json:
        print( "Field 'mural_link' not returned in result\nResult:"+ json.dumps( response_json, indent=3 ) )
        return None
    if "widgets_arr" not in response_json:
        print( "Field 'widgets_arr' not returned in result\nResult:"+ json.dumps( response_json, indent=3 ) )
        return None
    mural_id = response_json["mural_id"]
    mural_link = response_json["mural_link"]
    widgets_arr = response_json["widgets_arr"]
    html = HTML( "<h3>Mural for: Workshop Part 2 of 2</h3>" + 
             "<p>Mural ID: <code>" + mural_id + "</code></p>" +
             "<p>\"Visitor\" link: <a href='" + mural_link + "' target='_other'>Click to open mural in a new tab</a></p>" )
    display( html )
    return mural_id, mural_link, widgets_arr

In [ ]:
# Cell: 03

g_mural_id, g_mural_link, g_widgets_arr = setUpForPart2( g_room_name, g_workshop_key )

## 7.2 Put sticky note data into a DataFrame

In [ ]:
# Cell: 04

import pandas as pd

widgets_df_full = pd.DataFrame( g_widgets_arr )
g_stickies_df = widgets_df_full[ widgets_df_full["type"] == "sticky-note" ].copy()
g_stickies_df = g_stickies_df[["id","x", "y","width","height","text"]].reset_index( drop=True )
g_stickies_df

## 7.3 Run NLP analysis on sticky note comments

In [ ]:
# Cell: 05

import watson_nlp
from watson_nlp.toolkit import predict_document_sentiment

g_syntax_model = watson_nlp.load( watson_nlp.download( "syntax_izumo_en_stock" ) )
print( "Done" )

In [ ]:
# Cell: 06

g_sentiment_model = watson_nlp.load( watson_nlp.download( "sentiment_sentence-bert_multi_stock" ) )
print( "Done" )

In [ ]:
# Cell: 07

import re

def analyzeComment( comment ):
    syntax_result = g_syntax_model.run( comment )
    sentiment_result  = g_sentiment_model.run_batch( syntax_result.get_sentence_texts(), syntax_result.sentences )
    document_sentiment = predict_document_sentiment( sentiment_result, g_sentiment_model.class_idxs )
    sentiment_dict = document_sentiment.to_dict()
    sentiment_dict["label"] = re.sub( r"^.*_", "", sentiment_dict["label"].lower() ).title()
    return sentiment_dict

def generateSentimentColumns( row ):
    comment = row["text"]
    sentiment_dict = analyzeComment( comment )
    return sentiment_dict["label"], sentiment_dict["score"]

def getPOS( tokens_arr ):
    result = { "POS_NOUN" : [], "POS_VERB" : [], "POS_ADJ" : [], "POS_ADV" : [] }
    for token in tokens_arr:
        txt = token["lemma"] if token["lemma"] else token["span"]["text"].lower()
        pos = token["part_of_speech"]
        if pos in result.keys():
            result[ pos ].append( txt )
    return result

def generateSyntaxColumns( row ):
    comment = row["text"]
    syntax_dict = g_syntax_model.run( comment, parsers=( "token", "lemma", "part_of_speech" ) ).to_dict()
    pos = getPOS( syntax_dict["tokens"] )
    return pos["POS_NOUN"], pos["POS_ADJ"]

In [ ]:
# Cell: 08

g_stickies_w_sentiment_df = g_stickies_df.copy()
g_stickies_w_sentiment_df[ [ "SENTIMENT", "SENTIMENT_SCORE" ] ] = g_stickies_w_sentiment_df.apply ( generateSentimentColumns, axis=1, result_type="expand" )
g_stickies_w_pos_df = g_stickies_w_sentiment_df.copy()
g_stickies_w_pos_df[ [ "NOUNS", "ADJECTIVES" ] ] = g_stickies_w_pos_df.apply ( generateSyntaxColumns, axis=1, result_type="expand" )
g_stickies_w_pos_df

# Step 8: Customize NLP model for domain using dictionaries

The Watson NLP library provides a mechanism to easily create dictionaries of domain-specific terms and then extract entities from text using those dictionaries.

See: [Detecting entities with a custom dictionary](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp-entities-dict.html?context=cpdaas&audience=wdp)

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>8.1</b> Create a directory for custom dictionary files</li>
<li style="margin: 0px 0px 3px 0px;"><b>8.2</b> Create a dictionary file for lunch-related terms</li>
<li style="margin: 0px 0px 3px 0px;"><b>8.3</b> Create a dictionary file for golf-related terms</li>
<li style="margin: 0px 0px 3px 0px;"><b>8.4</b> Create custom dictionaries</li>
<li style="margin: 0px 0px 3px 0px;"><b>8.5</b> Test extracting entities using custom dictionaries</li>
</ol>

## 8.1 Create a directory for custom dictionary files

In [ ]:
# Cell: 09

import os
dictionaries_dir = "Custom_NLP" 
os.makedirs( dictionaries_dir, exist_ok=True )
print( "Done")

## 8.2 Create a dictionary file for lunch-related terms

In [ ]:
# Cell: 10

lunch_file = 'lunch.csv'
with open( os.path.join( dictionaries_dir, lunch_file ), 'w' ) as table:
    table.write( "\"label\", \"entry\"\n" )
    table.write( "\"LOCATION\", \"far\"\n" )
    table.write( "\"MENU\", \"vegetarian\"\n" )
    table.write( "\"SERVICE\", \"service\"\n" )
    table.write( "\"SERVICE\", \"slow\"\n" )
    table.write( "\"SERVICE\", \"waiter\"\n" )
    table.write( "\"FOOD\", \"food\"\n" )
    table.write( "\"FOOD\", \"bread\"\n" )
    table.write( "\"FOOD\", \"fresh\"\n" )
    table.write( "\"LUNCH\", \"lunch\"\n" )
print( "Done" )

In [ ]:
# Cell: 11

!cat Custom_NLP/lunch.csv

## 8.3 Create a dictionary file for golf-related terms

In [ ]:
# Cell: 12

golf_file = 'mini-golf.csv'
with open( os.path.join( dictionaries_dir, golf_file ), 'w' ) as table:
    table.write( "\"label\", \"entry\"\n" )
    table.write( "\"GOLF\", \"mini-golf\"\n" )
    table.write( "\"GOLF\", \"minigolf\"\n" )
    table.write( "\"GOLF\", \"golf\"\n" )
    table.write( "\"PLAY\", \"hole in one\"\n" )
    table.write( "\"WEATHER\", \"weather\"\n" )
    table.write( "\"WEATHER\", \"rain\"\n" )
    table.write( "\"COURSE\", \"golf course\"\n" )
    table.write( "\"COURSE\", \"course\"\n" )
print( "Done" )

## 8.4 Create custom dictionaries

In [ ]:
# Cell: 13

import watson_nlp

lunch_config = {
    "name"   : "lunch",
    "source" : lunch_file,
    "dict_type" : "table",
    "mappings": { "columns": [ "label", "entry" ], "entry" : "entry" },
    "consolidate" : "ContainedWithin",
    "case"        : "insensitive"
}

golf_config = {
    "name"   : "golf",
    "source" : golf_file,
    "dict_type" : "table",
    "mappings": { "columns": [ "label", "entry" ], "entry" : "entry" },
    "consolidate" : "ContainedWithin",
    "case"        : "insensitive"
}

dict_arr = watson_nlp.toolkit.DictionaryConfig.load_all( [ lunch_config, golf_config ] )

g_custom_dictionaries = watson_nlp.resources.feature_extractor.RBR.train( 
    dictionaries_dir,
    language = "en", 
    dictionaries = dict_arr
)

print( "Done" )

## 8.5 Test extracting entities using custom dictionaries

In [ ]:
# Cell: 14

import json

# Testing ...
comment = "The golf course was really great"
entities_result = g_custom_dictionaries.executor.get_raw_response( comment, language = "en" )
print( json.dumps( entities_result["annotations"], indent=3 ) )

# Step 9: Identify entities in sticky note comments

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>9.1:</b> Define function to extract entities in a DataFrame</li>
<li style="margin: 0px 0px 3px 0px;"><b>9.2:</b> Extract entities in the sticky notes DataFrame</li>
</ol>

## 9.1 Define function to extract entities in a DataFrame

In [ ]:
# Cell: 15

import re

def getTopEntities( entities_raw ):
    entities = {}
    dictionary_names = [ "LUNCH", "GOLF" ]
    for category in entities_raw["annotations"].keys():
        catgory_name = re.sub( r"^.+\_", "", category ).upper();
        if catgory_name not in entities:
            entities[ catgory_name ] = []
        for match in entities_raw["annotations"][category]:
            label = match["label"]
            if ( label not in dictionary_names ) and ( label not in entities[ catgory_name ] ):
                entities[ catgory_name ].append( label )
    top_category, top_entities = max( entities.items(), key = lambda x: len(set(x[1])) )
    result = { "DICTIONARY" : "", "ENTITIES" : [] }
    if len( top_entities ) > 0:
        result = { "DICTIONARY" : top_category, "ENTITIES" : top_entities }
    return result
    
def generateEntitiesColumns( row ):
    comment = row["text"]
    entities_raw = g_custom_dictionaries.executor.get_raw_response( comment, language = "en" )
    entities_dict = getTopEntities( entities_raw )
    return entities_dict["DICTIONARY"], entities_dict["ENTITIES"]

In [ ]:
# Cell: 16

test_df = g_stickies_w_pos_df[0:2].copy();
test_df

In [ ]:
# Cell: 17

test_df[ [ "DICTIONARY", "ENTITIES" ] ] = test_df.apply ( generateEntitiesColumns, axis=1, result_type="expand" )
test_df

## 9.2 Extract entities in the sticky notes DataFrame

In [ ]:
# Cell: 18

g_stickies_w_entities_df = g_stickies_w_pos_df.copy()
g_stickies_w_entities_df[ [ "DICTIONARY", "ENTITIES" ] ] = g_stickies_w_entities_df.apply ( generateEntitiesColumns, axis=1, result_type="expand" )
g_stickies_w_entities_df[ [ "text", "SENTIMENT", "SENTIMENT_SCORE", "NOUNS", "ADJECTIVES", "DICTIONARY", "ENTITIES" ] ]

# Step 10: Cluster sticky notes by entities

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>10.1</b> Define function to cluster sticky note comments by entity</li>
<li style="margin: 0px 0px 3px 0px;"><b>10.2</b> Visualize how clustering works</li>
<li style="margin: 0px 0px 3px 0px;"><b>10.3</b> Cluster sticky note comments by entity</li>
</ol>

## 10.1 Define function to cluster sticky note comments by entity

Build a *dendrogram* to perform *hierarchical clustering* of comments based on the extracted entities.

<!-- See: [scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) -->


In [ ]:
# Cell: 19

from collections import OrderedDict
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from scipy import cluster

def countWords( df, col_name, min_count ):
    all_words = {}
    for index, row in df.iterrows():
        words_arr = row[ col_name ]
        for word in words_arr:
            if word not in all_words:
                all_words[word] = 0
            all_words[word] += 1
    common_words = dict( [ (k,v) for k,v in all_words.items() if v > min_count ] )
    ordered_common_words = OrderedDict( sorted( common_words.items(), key=lambda x:x[1], reverse=True ) )
    return ordered_common_words

def uniqueWords( df, col_name, min_count ):
    words_od = countWords( df, col_name, min_count )
    unique_words = list( words_od.keys() )
    return sorted( unique_words )

def buildWordsMatrix( df, col_name, min_count ):
    unique_words = uniqueWords( df, col_name, min_count )
    labels = []
    matrix = []
    indices_org = []
    omitted_indices = []
    for index, df_row in df.iterrows():
        label_arr = []
        matrix_row = []
        for word in unique_words:
            if word in df_row[ col_name ]:
                label_arr.append( word )
                matrix_row.append( 1 )
            else:
                matrix_row.append( 0 )
        if( len( label_arr ) > 0 ):
            labels.append( " | ".join( label_arr ) )
            matrix.append( matrix_row )
            indices_org.append( index )
        else:
            omitted_indices.append( index )
    return labels, matrix, indices_org, omitted_indices, unique_words

def printExplanation( unique_labels, matrix, indices, df ):
    maxlen = len( max( unique_labels, key=len ) ) + 2
    print( "".join( [ label.rjust( maxlen ) for label in unique_labels ] ) )
    for i in range( len( matrix ) ):
        row = list( matrix[i] )
        row_str = "".join( [ str( entry ).rjust( maxlen ) for entry in row ] )
        print( row_str + "  " + df.loc[indices[i],"text"] )

def addClusterH( df, col_name, Z, labels, indices_org, omitted_indices ):
    cutree = cluster.hierarchy.cut_tree( Z, height=1.25 )
    cluster_arr = []
    for i in range( len(cutree) ):
        cluster_num = cutree[i][0]
        label = labels[i]
        index = indices_org[i]
        row = df.iloc[index]
        cluster_arr.append( list( row ) + [ cluster_num, label ] )
    other_cluster_num = int( max( cutree ) + 1 )
    for index in omitted_indices:
        row = df.iloc[index]
        cluster_arr.append( list( row ) + [ other_cluster_num, " | ".join( row[ col_name ] ) ] )
    clusterH_df = pd.DataFrame( cluster_arr, columns = df.columns.tolist() + [ "H_CLUSTER_ID", "LABEL" ] )
    return clusterH_df

def clusterH( df, col_name, explain=False ):
    labels, matrix, indices_org, omitted_indices, unique_words = buildWordsMatrix( df, col_name, 0 )
    if explain:
        printExplanation( unique_words, matrix, indices_org, df )
        return;
    Z = linkage( matrix, "single" )
    plt.figure( figsize=( 5, 8 ) )
    plt.gca().spines["left"].set_visible(False)
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["right"].set_visible(False)
    dend = dendrogram( Z,
                       get_leaves=True,
                       orientation="left",
                       labels=labels,
                       leaf_font_size=12,
                       show_leaf_counts=True)
    plt.show()
    clusterH_df = addClusterH( df, col_name, Z, labels, indices_org, omitted_indices )
    return clusterH_df.sort_values( [ "H_CLUSTER_ID", "LABEL" ], ignore_index=True )

## 10.2 Visualize how clustering works

In [ ]:
# Cell: 20

clusterH( g_stickies_w_entities_df, "ENTITIES", explain=True )

## 10.3 Cluster sticky note comments by entity

In [ ]:
# Cell: 21

g_stickies_w_cluster_df = clusterH( g_stickies_w_entities_df, "ENTITIES" )

In [ ]:
# Cell: 22

g_stickies_w_cluster_df.iloc[:,5:]

# Step 11: Add positional data to improve clustering

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>11.1</b> List shapes in the mural</li>
<li style="margin: 0px 0px 3px 0px;"><b>11.2</b> Identify the "Lunch" box and the "Mini-golf" box widgets</li>
<li style="margin: 0px 0px 3px 0px;"><b>11.3</b> Identify which box - "Lunch" or "Mini-golf" - a sticky note is closest to</li>
<li style="margin: 0px 0px 3px 0px;"><b>11.4</b> Cluster with entities and positional data</li>
</ol>

## 11.1 List shapes in the mural

In [ ]:
# Cell: 23

import requests

def refreshAccessToken( room_name, workshop_key ):
    url = "https://weavesphere-mural-oauth.tqns6lm651z.us-south.codeengine.appdomain.cloud/refresh-token"
    headers = { "Content-Type" : "application/json", "Accept" : "application/json" }
    data = json.dumps( { "room_name" : room_name, "workshop_key" : workshop_key } )
    response = requests.request( "POST", url, headers=headers, data=data )
    response_json = response.json()
    if "error_str" in response_json:
        print( response_json["error_str"] )
        return None
    if "access_token" not in response_json:
        print( "Field 'access_token' not returned in result" )
        return None
    return response_json["access_token"]

def listShapes( room_name, workshop_key, mural_id  ):
    mural_oauth_token = refreshAccessToken( room_name, workshop_key )
    if mural_oauth_token is None:
        return
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets?type=shapes"
    headers = { "Content-Type" : "application/json", "Accept": "application/json", "Authorization": "Bearer " + mural_oauth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = response.json()
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [ ]:
# Cell: 24

g_shapes_arr = listShapes( g_room_name, g_workshop_key, g_mural_id  )
print( json.dumps( g_shapes_arr, indent=3 ) )

## 11.2 Identify the "Lunch" box and the "Mini-golf" box widgets

Both the rectangle shape for the feedback comments about lunch and the one for comments about mini-golf had a title and a description added to the mural outline.

That means the JSON for those widgets has a non-empty value for `title` and `instruction`.

See: [Organize your mural with the outline feature](https://support.mural.co/en/articles/2113749-organize-your-mural-with-the-outline-feature)

In [ ]:
# Cell: 25

def getLunchAndMinigolfBoxes( widgets_arr ):
    lunch_widget = None
    golf_widget = None
    for widget in widgets_arr:
        widget_copy = { "id" : widget["id"],
                        "x"  : widget["x"],
                        "y"  : widget["y"],
                        "height" : widget["height"],
                        "width"  : widget["width"] }
        if ( "title" in widget ) and re.match( r".*lunch", widget["title"], re.IGNORECASE ):
            lunch_widget = widget_copy
        elif ( "title" in widget ) and re.match( r".*golf", widget["title"], re.IGNORECASE ):
            golf_widget = widget_copy
        if ( None != lunch_widget ) and ( None != golf_widget ):
            break;
    return lunch_widget, golf_widget

In [ ]:
# Cell: 26

g_lunch_widget, g_golf_widget = getLunchAndMinigolfBoxes( g_shapes_arr )
print( "Done" )

In [ ]:
# Cell: 27

print( json.dumps( { "lunch_widget" : g_lunch_widget, "golf_widget" : g_golf_widget }, indent=3 ) )

## 11.3 Identify which box - "Lunch" or "Mini-golf" - a sticky note is closest to

In [ ]:
# Cell: 28

def stickyCategory( row ):
    sticky_center_x = row["x"]  + ( 0.5 * row["width"] )
    lunch_center_x  = g_lunch_widget["x"] + ( 0.5 * g_lunch_widget["width"] )
    golf_center_x   = g_golf_widget["x"]  + ( 0.5 * g_golf_widget["width"] )
    lunch_distance  = abs( sticky_center_x - lunch_center_x )
    golf_distance   = abs( sticky_center_x - golf_center_x )
    return "LUNCH" if ( lunch_distance < golf_distance ) else "GOLF"

In [ ]:
# Cell: 29

row = g_stickies_w_cluster_df.iloc[0]
print( row["text"] )
stickyCategory( row )

In [ ]:
# Cell: 30

g_stickies_w_category_df = g_stickies_w_cluster_df.copy()
g_stickies_w_category_df["category_box"] = g_stickies_w_category_df.apply( stickyCategory, axis=1 )
g_stickies_w_category_df.iloc[:,5:]

### Clarity!

Now, even ambiguous comments, like "So fun!" and "Let's do it again!", have some context.  

Given their position on the mural, you at least know whether the comment is referring to lunch or mini-golf.  

You could not figure that out using only NLP.  Even a human couldn't guess what those comments were referring to.  But MURAL's positional information can tell us for sure.

## 11.4 Cluster with entities and positional data

In [ ]:
# Cell: 31

g_lunch_stickies_w_entities_df = g_stickies_w_category_df[ g_stickies_w_category_df["category_box"] == "LUNCH" ].copy().reset_index(drop=True).drop( [ "LABEL", "H_CLUSTER_ID" ], axis=1 )
g_golf_stickies_w_entities_df = g_stickies_w_category_df[ g_stickies_w_category_df["category_box"] == "GOLF" ].copy().reset_index(drop=True).drop( [ "LABEL", "H_CLUSTER_ID" ], axis=1 )
print( "Done" )

### Position: Lunch rectangle

In [ ]:
# Cell: 32

g_lunch_stickies_w_cluster_df = clusterH( g_lunch_stickies_w_entities_df, "ENTITIES" )

In [ ]:
# Cell: 33

g_lunch_stickies_w_cluster_df["LABEL"] = g_lunch_stickies_w_cluster_df["LABEL"].replace( "", "GENERAL" )
g_lunch_stickies_w_cluster_df.iloc[:,5:]

### Position: Golf rectangle

In [ ]:
# Cell: 34

g_golf_stickies_w_cluster_df = clusterH( g_golf_stickies_w_entities_df, "ENTITIES" )

In [ ]:
# Cell: 35

g_golf_stickies_w_cluster_df["LABEL"] = g_golf_stickies_w_cluster_df["LABEL"].replace( "", "GENERAL" )
g_golf_stickies_w_cluster_df.iloc[:,5:]

# Step 12: Visualize clusters in the mural

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>12.1:</b> Divide each category box into cluster zones</li>
<li style="margin: 0px 0px 3px 0px;"><b>12.2:</b> Define routines to add cluster label and move stickies</li>
<li style="margin: 0px 0px 3px 0px;"><b>12.3:</b> Move sticky notes into labeled clusters in the mural</li>
<li style="margin: 0px 0px 3px 0px;"><b>12.3:</b> [Optional] Routine for resetting the sticky notes</li>
</ol>

## 12.1 Divide each category box into cluster zones

In [ ]:
# Cell: 36

def getClusterZones( box, cluster_df ):
    title_padding = 450
    label_padding = 550
    padding = 50
    box_top = box["y"]
    box_height = box["height"]
    box_bottom = box_top + box_height
    box_left = box["x"]
    box_width = box["width"]
    box_right = box_left + box_width
    cluster_labels_df = cluster_df.loc[:,["LABEL","H_CLUSTER_ID"]].copy().drop_duplicates( subset=["LABEL","H_CLUSTER_ID"], keep="first")
    cluster_counts_df = cluster_df.loc[ :, [ "H_CLUSTER_ID" ] ].copy()
    cluster_counts_df["cluster_count"] = 0
    cluster_counts_df = cluster_counts_df.groupby( [ "H_CLUSTER_ID" ], as_index=False ).count()
    cluster_labels_df["cluster_count"] = cluster_labels_df.H_CLUSTER_ID.map( cluster_counts_df.set_index( "H_CLUSTER_ID" )["cluster_count"] )
    num_clusters = cluster_labels_df.shape[0]
    zone_height = ( box_bottom - box_top - title_padding ) / ( num_clusters - 0.25 )
    cluster_zones = {}
    zone_top = box_top + title_padding
    for index, row in cluster_labels_df.iterrows():
        label = row["LABEL"] if re.match( r"\S", row["LABEL"] ) else "GENERAL"
        cluster_zones[ label ] = { "x" : round( box_left + label_padding ), "y" : round( zone_top ), "height" : round( zone_height ), "width" : round( box_width ) }
        zone_top += zone_height
    return cluster_zones

### Lunch cluster zones

In [ ]:
# Cell: 37

lunch_cluster_zones = getClusterZones( g_lunch_widget, g_lunch_stickies_w_cluster_df )
print( json.dumps( lunch_cluster_zones, indent=3 ) )

### Golf cluster zones

In [ ]:
# Cell: 38

golf_cluster_zones = getClusterZones( g_golf_widget, g_golf_stickies_w_cluster_df )
print( json.dumps( golf_cluster_zones, indent=3 ) )

## 12.2 Define routines to add cluster label and move stickies

In [ ]:
# Cell: 39

from random import randrange

g_label_padding = 550
g_padding = 50

def addLabel( label, zone, mural_oauth_token ):
    # https://developers.mural.co/public/reference/createtextbox
    widget_data = { "x" : zone["x"] + g_padding - g_label_padding, 
                    "y" : zone["y"], 
                    "width" : 500,
                    "text" : "<b>" + label + "</b>",
                    "style": { "fontSize": 100 }
                  }
    url = "https://app.mural.co/api/public/v1/murals/" + g_mural_id + "/widgets/textbox"
    headers = { "Content-Type" : "application/json", "Accept" : "vnd.mural.preview", "Authorization" : "Bearer " + mural_oauth_token }
    data = json.dumps( widget_data )
    response = requests.request( "POST", url, headers=headers, data=data )
    response_json = response.json()
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    return msg

def moveSticky( sticky_id, x, y, mural_oauth_token ):
    # https://developers.mural.co/public/reference/updatestickynote
    url = "https://app.mural.co/api/public/v1/murals/" + g_mural_id + "/widgets/sticky-note/" + sticky_id
    headers = { "Content-Type" : "application/json", "Accept" : "application/json", "Authorization" : "Bearer " + mural_oauth_token }
    data = json.dumps( { "x" : x, "y" : y } )
    response = requests.request( "PATCH", url, headers=headers, data=data )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )

def lineUpInZone( zone, stickies_df, mural_oauth_token ):
    zone_padding = 0.05 * zone["height"]
    top_edge = zone["y"]
    left_edge = zone["x"] + zone_padding
    right_edge = left_edge
    for index, sticky in stickies_df.iterrows():
        sticky_margin = 0.1 * sticky["width"]
        random_range = round( 0.1 * sticky["width"] )
        if ( top_edge + sticky["height"] ) > ( zone["y"] + zone["height"] - zone_padding ):
            top_edge = zone["y"]
            left_edge = right_edge
        x = left_edge + 30 + randrange( random_range )
        if ( x + sticky["width"] ) > right_edge:
            right_edge = x + sticky["width"]
        y = top_edge + randrange( random_range ) + sticky_margin
        top_edge = y + sticky["height"]
        moveSticky( sticky["id"], x, y, mural_oauth_token )

## 12.3 Move sticky notes into labeled clusters in the mural

In [ ]:
# Cell: 40

import time
time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the lunch stickies move.
# ...

mural_oauth_token = refreshAccessToken( g_room_name, g_workshop_key )
if mural_oauth_token is not None:
    for label in lunch_cluster_zones.keys():
        zone = lunch_cluster_zones[ label ]
        error_msg = addLabel( label, zone, mural_oauth_token )
        if error_msg:
            print( "Error adding label: " + label + "\n" + error_msg )
            break
        stickies_df = g_lunch_stickies_w_cluster_df[ g_lunch_stickies_w_cluster_df["LABEL"] == label ].copy().reset_index( drop=True )
        error_msg = lineUpInZone( zone, stickies_df, mural_oauth_token )
        if error_msg:
            print( "Error moving stickies:\n" + error_msg )
            break

In [ ]:
# Cell: 41

time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the golf stickies move.
# ...

mural_oauth_token = refreshAccessToken( g_room_name, g_workshop_key )
if mural_oauth_token is not None:
    for label in golf_cluster_zones.keys():
        zone = golf_cluster_zones[ label ]
        error_msg = addLabel( label, zone, mural_oauth_token )
        if error_msg:
            print( "Error adding label: " + label + "\n" + error_msg )
            break
        stickies_df = g_golf_stickies_w_cluster_df[ g_golf_stickies_w_cluster_df["LABEL"] == label ].copy().reset_index( drop=True )
        error_msg = lineUpInZone( zone, stickies_df, mural_oauth_token )
        if error_msg:
            print( "Error moving stickies:\n" + error_msg )
            break

When you run the previous two cells, your mural should look something like this:

<img src="https://github.com/spackows/MURAL-API-Samples/blob/main/images/weavesphere-2022-cluster-stickies-in-mural.gif?raw=true" alt="Cluster stickies in mural" width="70%" />

## 12.4 [Optional] Routine for resetting the sticky notes

You can put the sticky notes back in their original position using the following routine.

Note: This routine doesn't remove the labels.  So if you want those removed too, you have to remove them by hand in mural.

In [ ]:
# Cell: 42

def resetStickyPositions( room_name, workshop_key ):
    mural_oauth_token = refreshAccessToken( room_name, workshop_key )
    if mural_oauth_token is None:
        return
    for index, row in g_stickies_w_pos_df.iterrows():
        moveSticky( row["id"], row["x"], row["y"], mural_oauth_token )

In [ ]:
# Cell: 43

resetStickyPositions( g_room_name, g_workshop_key )

# Step 13: Add a chart to the mural

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>13.1:</b> Shape entities data</li>
<li style="margin: 0px 0px 3px 0px;"><b>13.2:</b> Plot a chart in notebook</li>
<li style="margin: 0px 0px 3px 0px;"><b>13.3:</b> Save the plot as an image file in the local notebook working directory</li>
<li style="margin: 0px 0px 3px 0px;"><b>13.4:</b> Upload the plot image file to the mural</li>
</ol>

## 13.1  Shape entities data

In [ ]:
# Cell: 44

g_stickies_w_category_df["LABEL"] = g_stickies_w_category_df["LABEL"].replace( "", "GENERAL" )
theme_counts_df = g_stickies_w_category_df.loc[ :, [ "category_box", "LABEL" ] ].copy()
theme_counts_df["count"] = 0
theme_counts_df = theme_counts_df.groupby( [ "category_box", "LABEL" ], as_index=False ).count()
theme_counts_df.sort_values( [ "category_box", "count" ], inplace=True, ascending=[ False, True ], ignore_index=True )
theme_counts_df

In [ ]:
# Cell: 45

theme_sentiment_df = g_stickies_w_category_df[ g_stickies_w_category_df["SENTIMENT"] == "Positive" ].loc[ :, [ "category_box", "LABEL" ] ].copy()
theme_sentiment_df["num_positive"] = 0
theme_sentiment_df = theme_sentiment_df.groupby( [ "category_box", "LABEL" ], as_index=False ).count()
theme_sentiment_df.sort_values( [ "category_box", "num_positive", "LABEL" ], inplace=True, ascending=[ False, False, True ], ignore_index=True )
theme_sentiment_df

In [ ]:
# Cell: 46

themes_df = theme_counts_df.merge( theme_sentiment_df, how="left", on=[ "category_box", "LABEL" ] )
themes_df["num_positive"].fillna( 0, inplace=True ) 
themes_df["score"] = round( themes_df["num_positive"] / themes_df["count"], 2 )
themes_df.sort_values( [ "category_box", "score" ], ascending=[ False, False ], inplace=True, ignore_index=True )
themes_df

## 13.2  Plot a chart in notebook

In [ ]:
# Cell: 47

# This is needed for the hover text to work, but you need to use "%matplotlib inline" to turn off this mode
%matplotlib notebook 


my_palette = {
    # https://coolors.co/palette/ff595e-ffca3a-8ac926-1982c4-6a4c93
    "red"      : "#FF595E",
    "yellow"   : "#FFCA3A",
    "green"    : "#8AC926",
    "blue"     : "#1982C4",
    "purple"   : "#6A4C93"
}

from matplotlib.colors import LinearSegmentedColormap
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
import numpy as np

cmap = LinearSegmentedColormap.from_list( "rbg", [ my_palette["blue"], my_palette["green"], my_palette["yellow"] ], N = 256 ) 

fig_bar = plt.figure( figsize=( 14, 6 ) )
fig_bar.suptitle( "Sentiment by theme", fontsize = 22 )
plt.subplots_adjust( top = 0.7 )

gs = GridSpec( 7, 15, figure=fig_bar )
axs1 = fig_bar.add_subplot(gs[:4, 0:7])
axs2 = fig_bar.add_subplot(gs[:4, 8:13])
cb_axs = fig_bar.add_subplot(gs[6, 4:11])

axes = [ axs1, axs2 ]
bars = []
annotations = []

for category, axs in zip( themes_df["category_box"].unique(), axes ):
    
    labels = list( themes_df[ themes_df["category_box"] == category ].loc[ :, "LABEL" ] )
    counts = list( themes_df[ themes_df["category_box"] == category ].loc[ :, "count" ] )
    scores = list( themes_df[ themes_df["category_box"] == category ].loc[ :, "score" ] )
    num_bars  = len( labels )
    positions = np.arange( num_bars )
    colors = [ cmap( score ) for score in scores ]
    
    category_bars = axs.bar( labels, counts, color=colors )
    bars.append( category_bars )
    
    axs.set_yticks([])
    axs.set_ylabel( "Total comments", fontsize="9" )
    axs.tick_params( axis="x", labelsize=8)
    axs.set_title( category, fontsize=18, pad=23 )
    axs.spines["top"].set_visible( False )
    axs.spines["right"].set_visible( False )
    axs.spines["left"].set_visible( False )
    
    category_annotations = []
    for i in range( len( category_bars ) ):
        theme = labels[i]
        comments = list( g_stickies_w_category_df[ ( g_stickies_w_category_df["category_box"] == category ) & ( g_stickies_w_category_df["LABEL"] == theme ) ].loc[ :, "text" ] )
        annot = axs.annotate( "\n\n".join( comments ), xy=( 0.1, 0.8 ), fontsize="9", xycoords="axes fraction", verticalalignment="top" )
        annot.set_bbox( dict( facecolor="lightgrey", alpha=0.7, edgecolor="lightgrey") )
        annot.set_visible( False )
        category_annotations.append( annot )
    annotations.append( category_annotations )
    
def contains( bar, event ):
    x = event.xdata;
    y = event.ydata;
    if ( x >= bar.get_x() ) and ( x <= ( bar.get_x() + bar.get_width() ) ) and ( y >= bar.get_y() ) and ( y <= ( bar.get_y() + bar.get_height() ) ):
        return True
    return False

def hover( event ):
    for i in range( len( axes ) ):
        for j in range( len( bars[i] ) ):
            theme_bar = bars[i][j]
            theme_annot = annotations[i][j]
            if ( event.inaxes == axes[i] ) and contains( theme_bar, event ):
                theme_annot.set_visible( True )
            else:
                theme_annot.set_visible( False )

fig_bar.canvas.mpl_connect( "motion_notify_event", hover )

cmappable = ScalarMappable( norm=Normalize(0,1), cmap=cmap )
cbar = plt.colorbar( cmappable, cax=cb_axs, orientation="horizontal" )
cbar.set_ticks( [ 0.0, 0.5, 1.0 ] )
cbar.set_ticklabels( [ "Negative", "Neutral", "Positive" ] )
cb_axs.tick_params( axis="x", labelsize=10)
cb_axs.invert_xaxis()

plt.show()

In [ ]:
# Cell: 48

# This is needed to turn off the previous mode
%matplotlib inline

## 13.3  Save the plot as an image file in the local notebook working directory

In [ ]:
# Cell: 49

g_figure_file_name = "theme_sentiment_bars.png"

fig_bar.savefig( g_figure_file_name )

!ls -l

## 13.4  Upload the plot image file to the mural

In [ ]:
# Cell: 50

def refreshAccessToken( room_name, workshop_key ):
    url = "https://weavesphere-mural-oauth.tqns6lm651z.us-south.codeengine.appdomain.cloud/refresh-token"
    headers = { "Content-Type" : "application/json", "Accept" : "application/json" }
    data = json.dumps( { "room_name" : room_name, "workshop_key" : workshop_key } )
    response = requests.request( "POST", url, headers=headers, data=data )
    response_json = response.json()
    if "error_str" in response_json:
        print( response_json["error_str"] )
        return None
    if "access_token" not in response_json:
        print( "Field 'access_token' not returned in result" )
        return None
    return response_json["access_token"]


def createUploadURL( mural_oauth_token, mural_id ):
    # https://developers.mural.co/public/docs/how-to-upload-an-image-to-a-mural
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/assets"
    headers = { "Content-Type" : "application/json", "Accept" : "application/json", "Authorization" : "Bearer " + mural_oauth_token }
    data = json.dumps( { "fileExtension": "png" } )
    response = requests.request( "POST", url, headers=headers, data=data )
    response_json = response.json()
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None, None, None
    if "value" not in response_json:
        print( "No value returned" )
        return None, None, None
    if "url" not in response_json["value"]:
        print( "upload 'url' field not returned from MURAL" )
        return None, None, None
    if "name" not in response_json["value"]:
        print( "'name' field not returned from MURAL" )
        return None, None, None
    if "headers" not in response_json["value"]:
        print( "'headers' field not returned from MURAL" )
        return None, None, None
    return response_json["value"]["url"], response_json["value"]["name"], response_json["value"]["headers"]


def uploadImage( upload_url, upload_headers, img_file_name ):
    # https://developers.mural.co/public/docs/how-to-upload-an-image-to-a-mural
    #headers = { "x-ms-blob-type" : "BlockBlob" }
    with open( img_file_name, "rb" ) as payload:
        response = requests.request( "PUT", upload_url, headers=upload_headers, data=payload )
    #print( response.text )


def createImageWidget( mural_oauth_token, mural_id, storage_file_name ):
    # https://developers.mural.co/public/reference/createimage
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/image"
    headers = { "Content-Type" : "application/json", "Accept" : "application/json", "Authorization" : "Bearer " + mural_oauth_token }
    chart_width = 8000
    chart_height = 3200
    mural_default_width = 9216
    mural_middle = mural_default_width / 2
    chart_left = mural_middle - ( chart_width / 2 )
    data = json.dumps( { "x" : chart_left,
                         "y" : -1 * chart_height - 400,
                         "height" : chart_height,
                         "width"  : chart_width,
                         "name"   : storage_file_name } )
    response = requests.request( "POST", url, headers=headers, data=data )
    response_json = response.json()
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return
    print( "Done" )


def addImageToMural( room_name, workshop_key, mural_id, img_file_name ):
    mural_oauth_token = refreshAccessToken( room_name, workshop_key )
    if mural_oauth_token is None:
        return
    upload_url, upload_name, upload_headers = createUploadURL( mural_oauth_token, mural_id )
    if ( upload_url is None ) or ( upload_name is None ) or ( upload_headers is None ):
        return
    uploadImage( upload_url, upload_headers, img_file_name )
    createImageWidget( mural_oauth_token, mural_id, upload_name )

In [ ]:
# Cell: 51

time.sleep(5)

# Quick!  After running this cell, switch to your browser tab
# where the mural is to see the image of the chart get added.
# ...

addImageToMural( g_room_name, g_workshop_key, g_mural_id, g_figure_file_name )

After running the previous cell, you will see the chart in the mural:

<img src="https://github.com/spackows/MURAL-API-Samples/blob/main/images/weavesphere-2022-chart-in-mural.png?raw=true" alt="Workshop mural with a chart" width="60%" />